# 前処理

Step0
- ファイル名と日付を整合させる処理
- citycode = 0 / NaN 除去
- time / dailyid が無い行除去

Step1
- 大カテゴリ、中カテゴリ、小カテゴリに分解
- ３次メッシュ変換

Step２
- prefcodeから東京都を抽出
- citycodeより調布市を抽出
- 調布市外は OoR（Out of Range）= 11111111 に統一
- 15分ごとのサンプリング
- 居住地付与
    - 夜間時間帯（22:00–06:00）のメッシュIDを参考にする
    - メッシュ単位で夜間の累積滞在時間を集計する
    - 累積滞在時間が最大となるメッシュを 推定居住地（home） としPoIカラムにおいてhomeを付与




Step３
遷移確率算出の処理フロー

- dailyid × time_15min で並び替え，個人ごとの時系列を構築
- 状態を (mesh_level3, poi_large) と定義する
- 個人内で t → t+15分 の次状態を shift により生成
- 15分連続でない遷移（欠測・日跨ぎ）を除外
- time_15min × 状態 × 次状態 ごとに遷移回数を集計
- 遷移元状態ごとに正規化し，遷移確率を算出

In [ ]:
from pathlib import Path
import pandas as pd

# 検証で必要
import random
import numpy as np

pd.set_option('display.max_rows', 100)


## データを読み込む

In [ ]:
# 作成するファイルの日付　指定
file_date = "20190209"

# データを読み込む
trace_path = Path(
    f"/Users/y-osamu/study/poi_sim/data/raw/trace/sktrace(old)/{file_date}.csv"
)
trace_df = pd.read_csv(trace_path,index_col=0)
trace_df = trace_df.reset_index()
trace_df = trace_df.drop(columns=['Unnamed: 0'])

display(trace_df.head())
print(f"レコード数: {len(trace_df)}")


## Step 0
- ファイル名と日付を整合させる処理
- citycode = 0 / NaN 除去
- time / dailyid が無い行除去

In [ ]:
# ファイル名と日付を整合させる処理
file_date_name = pd.to_datetime(trace_path.stem, format="%Y%m%d")
print(f"ファイルの日付: {file_date_name.date()}")

trace_df["time"] = pd.to_datetime(trace_df["time"])
trace_df["time"] = file_date_name + (trace_df["time"] - trace_df["time"].dt.normalize())

# citycode が 0 / NaN の行を除去
trace_df = trace_df.loc[
    trace_df["citycode"].notna() & (trace_df["citycode"] != 0)
].copy()

# dailyid が NaN / 空の行を除去
trace_df = trace_df.loc[
    trace_df["dailyid"].notna()
].copy()

# time が NaT の行を除去（念のため）
trace_df = trace_df.loc[
    trace_df["time"].notna()
].copy()

# インデックスを振り直す
trace_df = trace_df.reset_index(drop=True)


display(trace_df.head())


## Step1
- 大カテゴリ、中カテゴリ、小カテゴリに分解
- ３次メッシュ変換

In [ ]:
# PoIカテゴリを分解
## PoIのカテゴリが2回入っているので1回にまとめる　
### 加工前の状態［大カテゴリ、中カテゴリ、小カテゴリ、大カテゴリ、中カテゴリ、小カテゴリ］

def parse_poi(x):
    # ["['A', 'B', 'C']"] の形を想定
    if isinstance(x, list):
        x = x[0]

    # 前後の [ ] を除去
    x = x.strip()
    x = x.lstrip('[').rstrip(']')

    # カンマで分割してクォート除去
    return [v.strip().strip("'").strip('"') for v in x.split(',')]


trace_df['poi'] = trace_df['poi'].apply(parse_poi)
trace_df['poi'] = trace_df['poi'].apply(lambda x: x[:3])


# poi を必ず長さ3に正規化（move / OoR 対応）
def expand_poi_to_3(x):
    if not isinstance(x, list) or len(x) == 0:
        return ["OoR", "OoR", "OoR"]
    if len(x) == 1:
        return [x[0], x[0], x[0]]
    if len(x) == 2:
        return [x[0], x[1], x[0]]
    return x[:3]


trace_df["poi"] = trace_df["poi"].apply(expand_poi_to_3)


display(trace_df[['poi']].head())

In [ ]:
# PoIカテゴリをDataFrameに展開

trace_df[['poi_large', 'poi_middle', 'poi_small']] = pd.DataFrame(
    trace_df['poi'].tolist(),
    index=trace_df.index
)
display(trace_df.head())

# 各PoI列のユニーク数
print("largeカテゴリのユニーク数:", trace_df['poi_large'].nunique())
print("middleカテゴリのユニーク数:", trace_df['poi_middle'].nunique())
print("smallカテゴリのユニーク数:", trace_df['poi_small'].nunique())

# PoI Largeのカテゴリ一覧
large_categories = trace_df['poi_large'].unique()
print("Largeカテゴリ一覧:")
for category in large_categories:
    print("-", category)

# PoI Largeの”NA”カテゴリのデータ数
na_large_count = len(trace_df[trace_df['poi_large'] == 'NA'])
print(f"LargeカテゴリのNAのデータ数: {na_large_count}")

# NAカテゴリのデータを除去
trace_df = trace_df[trace_df['poi_large'] != 'NA']
print(f"NAカテゴリ除去後のレコード数: {len(trace_df)}")

In [ ]:
# ３次メッシュに変換
trace_df["mesh_level3"] = trace_df["mesh100mid"].astype(str).str[:-2]

display(trace_df.head(5))

In [ ]:
# step1のデータ保存
step1_path = Path(f"/Users/y-osamu/study/poi_sim/data/processed/yamada_processed/traj_normalized/step1_trace_{file_date}.csv")

trace_df.to_csv(step1_path, index=False)
print(f"Step1データを保存しました: {step1_path}")

## Step２
- prefcodeから東京都を抽出 citycodeより調布市を抽出
- 調布市に出現した dailyid を取得　その dailyid を持つ「全データ」を取得
- 調布市外は OoR（Out of Range）= 11111111 に統一
- 居住地付与
    - 軌跡データに含まれる推定居住地メッシュを home 
    - 深夜帯（0–5時）の滞在履歴を集計、最長滞在メッシュを home として付与
    - 対応表の作成、数値での管理

In [ ]:
# 保存したデータを確認
step1_path = Path(f"/Users/y-osamu/study/poi_sim/data/processed/yamada_processed/traj_normalized/step1_trace_{file_date}.csv")
trace_df = pd.read_csv(step1_path)

display(trace_df.head(2))   


In [ ]:
# prefcodeより東京都、citycodeより調布市のデータを取得する
## JIS市区町村コード：13208（参考：東京都は 13、調布市はその中の 208）
trace_chofu_stay = trace_df[
    (trace_df["citycode"] == 13208) &
    (trace_df["judge"] == "stay")
]

# stay したユーザの dailyid
chofu_stay_dailyids = trace_chofu_stay["dailyid"].unique()

# そのユーザの全履歴を取得
trace_chofu_users = trace_df[
    trace_df["dailyid"].isin(chofu_stay_dailyids)
].copy()

display(trace_chofu_users.head())
print(f"調布市に滞在・通過したユーザのレコード数: {len(trace_chofu_users)}")

In [ ]:
# trace_chofu_usersのデータの中で調布市外は mesh_level3をOoR（Out of Range）= 11111111 に統一   
CHOFU_CITY = 13208

trace_chofu_users.loc[
    trace_chofu_users["citycode"] != CHOFU_CITY,
    "mesh_level3"
] = "11111111"
trace_chofu_users["mesh_level3"].value_counts().head(10)



# 調布市のメッシュIDを持つが PoI Large が OoR のデータを除去
trace_chofu_users = trace_chofu_users.loc[
    ~(
        (trace_chofu_users["mesh_level3"] != "11111111") &
        (trace_chofu_users["poi_large"] == "OoR")
    )
].copy()

pd.set_option("display.max_rows", None)

# mesh_level3 と poi_large のクロス集計
pd.crosstab(
    trace_chofu_users["mesh_level3"],
    trace_chofu_users["poi_large"]
)


# mesh_level3 が 11111111 の行の poi_large, poi_middle, poi_small をすべて "OoR" に統一
mask_oor = trace_chofu_users["mesh_level3"] == "11111111"
trace_chofu_users.loc[mask_oor, "poi_large"] = "OoR"
trace_chofu_users.loc[mask_oor, "poi_middle"] = "OoR"
trace_chofu_users.loc[mask_oor, "poi_small"]  = "OoR"

trace_chofu_users.loc[
    trace_chofu_users["mesh_level3"] == "11111111",
    "poi_large"
].value_counts()



In [ ]:
TAU_MIN = 45
NON_POI = {"move", "OoR"}

# =========================
# 1. 15分離散化
# =========================
trace_15_df = trace_chofu_users.copy()
trace_15_df["time"] = pd.to_datetime(trace_15_df["time"], errors="coerce")
trace_15_df["time_15min"] = trace_15_df["time"].dt.floor("15min")

# =========================
# 2. dailyid × time_15min の完全グリッド
# =========================
grid = (
    trace_15_df
    .groupby("dailyid")["time_15min"]
    .agg(["min", "max"])
    .reset_index()
)

grid["time_15min"] = grid.apply(
    lambda r: pd.date_range(r["min"], r["max"], freq="15min"),
    axis=1
)
grid = grid.explode("time_15min").drop(columns=["min", "max"])

# =========================
# 3. merge（欠測生成）
# =========================
trace_15_df = (
    grid
    .merge(trace_15_df, on=["dailyid", "time_15min"], how="left")
    .sort_values(["dailyid", "time_15min"])
    .reset_index(drop=True)
)

# =========================
# 4. 連続滞在補完
# =========================
# 前の PoI（階層ごと）
trace_15_df["prev_l"] = trace_15_df.groupby("dailyid")["poi_large"].ffill()
trace_15_df["prev_m"] = trace_15_df.groupby("dailyid")["poi_middle"].ffill()
trace_15_df["prev_s"] = trace_15_df.groupby("dailyid")["poi_small"].ffill()

# 次の large（判定用）
trace_15_df["next_l"] = trace_15_df.groupby("dailyid")["poi_large"].bfill()

# 時刻差（分）
trace_15_df["gap_prev"] = (
    trace_15_df.groupby("dailyid")["time_15min"]
    .diff().dt.total_seconds().div(60)
)
trace_15_df["gap_next"] = (
    trace_15_df.groupby("dailyid")["time_15min"]
    .diff(-1).abs().dt.total_seconds().div(60)
)

# 補完条件
mask_fill = (
    trace_15_df["poi_large"].isna()
    & (trace_15_df["prev_l"] == trace_15_df["next_l"])
    & (~trace_15_df["prev_l"].isin(NON_POI))
    & (trace_15_df["gap_prev"] <= TAU_MIN)
    & (trace_15_df["gap_next"] <= TAU_MIN)
)

# ★ 正しい補完（3階層同時）
trace_15_df.loc[mask_fill, "poi_large"]  = trace_15_df.loc[mask_fill, "prev_l"]
trace_15_df.loc[mask_fill, "poi_middle"] = trace_15_df.loc[mask_fill, "prev_m"]
trace_15_df.loc[mask_fill, "poi_small"]  = trace_15_df.loc[mask_fill, "prev_s"]

# =========================
# 5. 未補完は move
# =========================
mask_na = trace_15_df["poi_large"].isna()
trace_15_df.loc[mask_na, ["poi_large", "poi_middle", "poi_small"]] = "move"

# 正規化（必須）
mask_move = trace_15_df["poi_large"] == "move"
trace_15_df.loc[mask_move, ["poi_middle", "poi_small"]] = "move"

mask_oor = trace_15_df["poi_large"] == "OoR"
trace_15_df.loc[mask_oor, ["poi_middle", "poi_small"]] = "OoR"

# =========================
# 6. 一時カラム削除
# =========================
trace_15_df = trace_15_df.drop(
    columns=["prev_l", "prev_m", "prev_s", "next_l", "gap_prev", "gap_next"],
    errors="ignore"
)

display(trace_15_df.head(5))
print(f"サンプリング後のレコード数: {len(trace_15_df)}")


- 居住地付与
    - 夜間時間帯（22:00–06:00）のメッシュIDを参考にする
    - メッシュ単位で夜間の累積滞在時間を集計する
    - 累積滞在時間が最大となるメッシュを 推定居住地（home） としPoIカラムにおいてhomeを付与



homeと決定された時間の分布を見る


In [ ]:
def assign_home_poi(
    trace_15_df: pd.DataFrame,
    stay_min: int = 15,
    night_start: int = 22,
    night_end: int = 6,
) -> pd.DataFrame:
    """
    夜間滞在時間最大メッシュを home として推定し，
    PoI カラムに home を付与した新しい DataFrame を返す．
    （元の trace_15_df は一切変更しない）
    """

    # =========================
    # 0. copy（元データ保護）
    # =========================
    df = trace_15_df.copy()

    # =========================
    # 1. 夜間データ抽出（必要最小限）
    # =========================
    night_df = df.loc[
        (df["time_15min"].dt.hour >= night_start)
        | (df["time_15min"].dt.hour < night_end),
        ["dailyid", "mesh_level3"]
    ].dropna(subset=["mesh_level3"])

    # =========================
    # 2. 夜間の累積滞在時間集計
    # =========================
    night_stay = (
        night_df
        .groupby(["dailyid", "mesh_level3"], as_index=False)
        .size()
        .rename(columns={"size": "n_slot"})
    )
    night_stay["stay_min"] = night_stay["n_slot"] * stay_min

    # =========================
    # 3. 推定居住地（最大滞在メッシュ）
    # =========================
    home_mesh_df = (
        night_stay
        .sort_values(["dailyid", "stay_min"], ascending=[True, False])
        .drop_duplicates("dailyid")
        .rename(columns={"mesh_level3": "home_mesh"})
        [["dailyid", "home_mesh"]]
    )

    # =========================
    # 4. home を PoI に反映（df のみ）
    # =========================
    df = df.merge(home_mesh_df, on="dailyid", how="left")

    mask_home = df["mesh_level3"] == df["home_mesh"]
    df.loc[mask_home, ["poi_large", "poi_middle", "poi_small"]] = "home"

    # 後片付け（内部列を残さない）
    df = df.drop(columns=["home_mesh"], errors="ignore")

    return df

trace_15_df_home = assign_home_poi(trace_15_df)
display(trace_15_df_home.head())
trace_15_df_home["poi_large"].value_counts()


In [ ]:
# home の分布を確認

stay_min = 15
night_start = 22
night_end = 6

# 夜間データ抽出
night_df = trace_15_df.loc[
    (trace_15_df["time_15min"].dt.hour >= night_start)
    | (trace_15_df["time_15min"].dt.hour < night_end),
    ["dailyid", "mesh_level3"]
].dropna(subset=["mesh_level3"])

display(night_df.head())
# # 夜間滞在時間集計
# night_stay = (
#     night_df
#     .groupby(["dailyid", "mesh_level3"], as_index=False)
#     .size()
#     .rename(columns={"size": "n_slot"})
# )
# night_stay["stay_min"] = night_stay["n_slot"] * stay_min

# home_mesh_df = (
#     night_stay
#     .sort_values(["dailyid", "stay_min"], ascending=[True, False])
#     .drop_duplicates("dailyid")
#     .rename(columns={"mesh_level3": "home_mesh"})
#     [["dailyid", "home_mesh", "stay_min"]]
# )

# home_mesh_df["stay_min"].max()



In [ ]:
# step2のデータ保存
step2_path = Path(f"/Users/y-osamu/study/poi_sim/data/processed/yamada_processed/traj_scoped/step2_trace_{file_date}.csv")
trace_15_df_home.to_csv(step2_path, index=False)
print(f"Step2データを保存しました: {step2_path}")

## Step３

遷移確率算出の処理フロー（trace_15_df → 遷移確率）

- dailyid × time_15min で並び替え，個人ごとの時系列を構築
- 状態を (mesh_level3, poi_large) と定義する
- 個人内で t → t+15分 の次状態を shift により生成
- 15分連続でない遷移（欠測・日跨ぎ）を除外
- time_15min × 状態 × 次状態 ごとに遷移回数を集計
- 遷移元状態ごとに正規化し，遷移確率を算出

In [ ]:
# 保存したデータを確認
step2_path = Path(f"/Users/y-osamu/study/poi_sim/data/processed/yamada_processed/traj_scoped/step2_trace_{file_date}.csv")
trace_15_df = pd.read_csv(step2_path)

display(trace_15_df.head(2))   


In [ ]:
# =========================
# Step 0: 前処理
# =========================
df = (
    trace_15_df
    .sort_values(["dailyid", "time_15min"])
    .copy()
)

df["time_15min"] = pd.to_datetime(df["time_15min"], errors="coerce")

# =========================
# Step 1: 状態定義
# =========================
df["state_mesh"] = df["mesh_level3"]
df["state_poi"]  = df["poi_large"]

# =========================
# Step 2: 次状態生成（個人内）
# =========================
df["next_mesh"] = df.groupby("dailyid")["state_mesh"].shift(-1)
df["next_poi"]  = df.groupby("dailyid")["state_poi"].shift(-1)
df["next_time"] = df.groupby("dailyid")["time_15min"].shift(-1)


# =========================
# Step 3: 遷移人数集計（★保存対象・十分統計量）
# =========================
transition_count_df = (
    df
    .loc[df["next_time"].notna()]  # 最終時刻だけ除外
    .groupby(
        [
            "time_15min",
            "state_mesh",
            "state_poi",
            "next_mesh",
            "next_poi",
        ],
        as_index=False
    )
    .agg(n_transition=("dailyid", "count"))
)



In [ ]:
pd.set_option("display.max_rows", 5)
display(transition_count_df)

In [ ]:
# =========================
# シミュレーション用データ構造作成
    # 正解滞在分布　メッシュ・PoIごと時間帯別エージェント人数分布
# =========================

out_path = Path(
    f"/Users/y-osamu/study/poi_sim/data/processed/yamada_processed/trans_count/state_count_{file_date}.parquet"
)


transition_count_df.to_csv(out_path,index=False)
print(f"状態人数分布データを保存しました: {out_path}")
